# CSK constellation optimization

* $\alpha \in \mathbb{R}^{N\times 3}$ is the set of input points, N vectors of length 3 (r,g,b)
* $H \in \mathbb{R}^{3\times 3}$ is the transformation matrix


The problem to solve is:
$$
\begin{aligned}
\max \min_{\alpha} \quad & ||\alpha H_{i,:} - \alpha H_{j,:}||_{2}^{2}, i=1...N, j=1...N, i\ne j\\
\textrm{s.t.} \quad & \sum_{j=1}^{3}\alpha_{i,j} \le 1, i=1...N\\
  &\alpha >= 0    \\
\end{aligned}
$$


This $\max \min$ problem can be rewritten as:

$$
\begin{aligned}
\max_{\alpha} \quad & Z\\ 
\textrm{s.t.} \quad & \sum_{j=1}^{3}\alpha_{i,j} \le 1, i=1...N\\
  &\alpha >= 0    \\
  & Z \le ||\alpha H_{1,:} - \alpha H_{2,:}||_{2}^{2}\\
  & Z \le ||\alpha H_{1,:} - \alpha H_{3,:}||_{2}^{2}\\
  & \vdots
\end{aligned}
$$

---
Install and import required packages

In [1]:
using Pkg
Pkg.add("JuMP")        # to express optimization problems
Pkg.add("Ipopt")       # solver
Pkg.add("PlotlyJS")    # for plotting
Pkg.add("DataFrames")

Pkg.status()

using JuMP
using Ipopt
using PlotlyJS
using DataFrames

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Status `~/.julia/environments/v1.9/Project.toml`
  [a93c6f00] DataFrames v1.6.1
  [7073ff75] IJulia v1.24.2
  [b6b21f68] Ipopt v1.4.2
  [4076af6c] JuMP v1.15.1
  [f0f68f2c] PlotlyJS v0.18.10
  [0f1e0344] WebIO v0.8.21


WebIO._IJuliaInit()


---
## Implementation
Number of points in the constellation

In [7]:
N = 4
M = 2

# Define an empty model and assign the solver.
# **TODO: experiment with different NLP [solvers](https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers)**
model = Model(Ipopt.Optimizer)

# Define the variable $\alpha$. It is very convenient that it can be defined as a $N\times 3$ matrix.
if M == 3
    @variable(model, ɑ[1:N, 1:M]  >= 0.0)
elseif M == 2
    @variable(model, ɑ[1:N, 1:M])
end

# Defining constraints
function line_equation(x1, y1, x2, y2)
    A = y2 - y1
    B = x1 - x2
    C = x2 * y1 - x1 * y2
    
    # Output the equation in the form Ax + By + C = 0
    # println("The general equation of the line is: $A*x + $B*y + $C = 0")
    return [A;B;C]
end

if M == 3
    @constraint(model, sum(ɑ, dims=2) .- 1 .== 0)
elseif M == 2
    vertex = [0 sqrt(3)/3; -1/2 -sqrt(3)/6; 1/2 -sqrt(3)/6]
    coeffs_1 = line_equation(vertex[1,1], vertex[1,2], vertex[2,1], vertex[2,2])
    coeffs_2 = line_equation(vertex[1,1], vertex[1,2], vertex[3,1], vertex[3,2])
    coeffs_3 = line_equation(vertex[2,1], vertex[2,2], vertex[3,1], vertex[3,2])
    
    println(coeffs_1)
    println(coeffs_2)
    println(coeffs_3)
    
    @constraint(model, constraint1, ɑ * coeffs_1[1:2] .+ coeffs_1[3] .>= 0)
    @constraint(model, constraint2, ɑ * coeffs_2[1:2] .+ coeffs_2[3] .>= 0)
    @constraint(model, constraint3, ɑ * coeffs_3[1:2] .+ coeffs_3[3] .<= 0)
end

[-0.8660254037844386, 0.5, -0.28867513459481287]
[-0.8660254037844386, -0.5, 0.28867513459481287]
[0.0, -1.0, -0.28867513459481287]


4-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 constraint3 : -ɑ[1,2] ≤ 0.28867513459481287
 constraint3 : -ɑ[2,2] ≤ 0.28867513459481287
 constraint3 : -ɑ[3,2] ≤ 0.28867513459481287
 constraint3 : -ɑ[4,2] ≤ 0.28867513459481287

**TODO: Explore with different norms. Using the Euclidean norm results in  a solver error.**

In [8]:
# Defining the transformation matrix of the channel
# **TODO: try with different $H$ matrices**
H = 55.3e-3*[1 0.042  0.030; 0.194  0.665  0.277;0.009 0.084 0.421]

# Defining the distance for the objective functions
manhattan(x,y) = sum((abs(x[i]-y[i]) for i in 1:M))
euclid(x,y) = sqrt(sum((x[i]-y[i])^2 for i in 1:M))
# norm = euclid # for the time-being only the Manhattan norm is working
norm = manhattan # for the time-being only the Manhattan norm is working

# Defining the variable with the transformation of the channel

# ɑH = ɑ*H  # define output constellation
ɑH = ɑ  # define output constellation

# calculate pairwise distances
dist_ = [norm(ɑH[i,:], ɑH[j,:]) for i in 1:N for j in 1:N if i != j]
# min_dist_ = minimum(dist_)
Nd = size(dist_)[1]
# @expression(model, dist[i=1:Nd], dist_[i])
@expression(model, min_dist, minimum(dist_)) 

min(min(min(min(min(min(min(min(min(min(min(abs(ɑ[1,1] - ɑ[2,1]) + abs(ɑ[1,2] - ɑ[2,2]), abs(ɑ[1,1] - ɑ[3,1]) + abs(ɑ[1,2] - ɑ[3,2])), abs(ɑ[1,1] - ɑ[4,1]) + abs(ɑ[1,2] - ɑ[4,2])), abs(ɑ[2,1] - ɑ[1,1]) + abs(ɑ[2,2] - ɑ[1,2])), abs(ɑ[2,1] - ɑ[3,1]) + abs(ɑ[2,2] - ɑ[3,2])), abs(ɑ[2,1] - ɑ[4,1]) + abs(ɑ[2,2] - ɑ[4,2])), abs(ɑ[3,1] - ɑ[1,1]) + abs(ɑ[3,2] - ɑ[1,2])), abs(ɑ[3,1] - ɑ[2,1]) + abs(ɑ[3,2] - ɑ[2,2])), abs(ɑ[3,1] - ɑ[4,1]) + abs(ɑ[3,2] - ɑ[4,2])), abs(ɑ[4,1] - ɑ[1,1]) + abs(ɑ[4,2] - ɑ[1,2])), abs(ɑ[4,1] - ɑ[2,1]) + abs(ɑ[4,2] - ɑ[2,2])), abs(ɑ[4,1] - ɑ[3,1]) + abs(ɑ[4,2] - ɑ[3,2]))

There are two different ways (at least) to formulate the optimization problem
* Directly formulating the $\max \min$ problem
* Rewriting it as a $\max$ problem with a slack variable and additional constraints

Only the latter one is working

**TODO: investigate why**

In [9]:
approx = true
if M == 3
	if approx
		# uses the rewriting of the maxmin
		@variable(model, Z >= 0)
		@constraint(model, Z .<= dist)
		@objective(model, Max, Z)
	else
		# this approch is currently not working. TODO: investigate
		f(args...) = min(args...)
		@objective(model, Max, f(dist...))
	end
	
elseif M ==2
	@objective(model, Max, min_dist)
end

min(min(min(min(min(min(min(min(min(min(min(abs(ɑ[1,1] - ɑ[2,1]) + abs(ɑ[1,2] - ɑ[2,2]), abs(ɑ[1,1] - ɑ[3,1]) + abs(ɑ[1,2] - ɑ[3,2])), abs(ɑ[1,1] - ɑ[4,1]) + abs(ɑ[1,2] - ɑ[4,2])), abs(ɑ[2,1] - ɑ[1,1]) + abs(ɑ[2,2] - ɑ[1,2])), abs(ɑ[2,1] - ɑ[3,1]) + abs(ɑ[2,2] - ɑ[3,2])), abs(ɑ[2,1] - ɑ[4,1]) + abs(ɑ[2,2] - ɑ[4,2])), abs(ɑ[3,1] - ɑ[1,1]) + abs(ɑ[3,2] - ɑ[1,2])), abs(ɑ[3,1] - ɑ[2,1]) + abs(ɑ[3,2] - ɑ[2,2])), abs(ɑ[3,1] - ɑ[4,1]) + abs(ɑ[3,2] - ɑ[4,2])), abs(ɑ[4,1] - ɑ[1,1]) + abs(ɑ[4,2] - ɑ[1,2])), abs(ɑ[4,1] - ɑ[2,1]) + abs(ɑ[4,2] - ɑ[2,2])), abs(ɑ[4,1] - ɑ[3,1]) + abs(ɑ[4,2] - ɑ[3,2]))

In [5]:
# Defining the models and getting the results
optimize!(model)
solution_summary(model)
input = value.(ɑ)
# output = input * H


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.13, running with linear solver MUMPS 5.6.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       80
Number of nonzeros in Lagrangian Hessian.............:      528

Total number of variables............................:       32
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

16×2 Matrix{Float64}:
 -0.402227  0.176289
 -0.369121  0.239724
 -0.398258  0.24273
 -0.366135  0.180187
 -0.386042  0.223381
 -0.345916  0.168854
 -0.353962  0.223083
 -0.387303  0.192941
 -0.421821  0.252192
 -0.329137  0.209262
 -0.338116  0.238849
 -0.35649   0.26008
 -0.28168   0.321068
 -0.433901  0.144127
 -0.415141  0.157585
 -0.3702    0.207672

## Plot results

Plot input constellation

In [6]:
df_i = DataFrame(input, :auto)
if M == 3
    plot(df_i, x=:x1, y=:x2, z=:x3, type="scatter3d", mode="markers")
elseif  M == 2
    # Assuming you have a DataFrame df_i with columns x1 and x2
    plot(df_i, x=:x1, y=:x2, type="scatter", mode="markers")
end

data: [
  "scatter with fields mode, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

plot output constellation

In [ ]:
df_out = DataFrame(output, :auto)
plot(df_out, x=:x1, y=:x2, z=:x3, type="scatter3d", mode="markers")

Verify Euclidean distance for input and output points:

In [ ]:
# euclid_norm  = [euclid(output[i,:], output[j,:]) for i in 1:N for j in 1:N if i != j];
euclid_norm  = [euclid(input[i,:], input[j,:]) for i in 1:N for j in 1:N if i != j];
min(euclid_norm...)

In [ ]:
plot(euclid_norm)

In [ ]:
plot(DataFrame([euclid_norm], :auto), x=:x1, kind="histogram")

In [ ]:
euclid_norm  = [euclid(input[i,:], input[j,:]) for i in 1:N for j in 1:N if i != j];
min(euclid_norm...)